# Import Libraries

In [ ]:
import joblib, pickle
import tweepy as tw
from textblob import TextBlob,Word
from TSA_pipeline import *
import snscrape.modules.twitter as sntwitter
import GetOldTweets3 as got
from datetime import datetime
from matplotlib.dates import date2num
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
consumer_key = "d2hXn9AgclF7xGx9u3QtAw4x0"
consumer_secret = "O2u1AAst69VKOgKDhRstyLpjGpaYIwxsCSTiLeON4pACtZRbMy"
access_token = "1491103209829679106-61ySETQ4bFkMnnDy23HBy68HTPvxUY"
access_secret = "H9jOC3g5NstrawQBG61J5nOB96FdpwLqbAx7YG2tZYJn2"
auth = tw.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)
api = tw.API(auth,wait_on_rate_limit = False)

In [ ]:
# Boris Johnson
id = 1223365339494453248
tweets = {}
df_tweets = pd.DataFrame(columns=['text','Timestamp','id'])
for index in range(5):
    # USER SEARCH
    handler = "BorisJohnson"
    res = api.user_timeline(screen_name=handler,count=1000,max_id=id)

    tweets = [{'text':tw.text,'Timestamp':tw.created_at, 'id':tw.id} for tw in res]
    df_tweets = df_tweets.append(pd.DataFrame.from_dict(tweets))
    id = tweets[len(tweets)-2]['id']
    print(id)
    tweets={}
    
df_tweets['sentiment'] = df_tweets['text'].apply(lambda x: TextBlob(x).sentiment[0])
df_tweets['subjectivity'] = df_tweets['text'].apply(lambda x: TextBlob(x).sentiment[1])

In [ ]:
# Jeremy Corbyn
id = 1222175012838412288
tweets = {}
df_tweets = pd.DataFrame(columns=['text','Timestamp','id'])
for index in range(5):
    # USER SEARCH
    handler = "jeremycorbyn"
    res = api.user_timeline(screen_name=handler,count=1000,max_id=id)

    tweets = [{'text':tw.text,'Timestamp':tw.created_at, 'id':tw.id} for tw in res]
    df_tweets = df_tweets.append(pd.DataFrame.from_dict(tweets))
    id = tweets[len(tweets)-2]['id']
    print(id)
    tweets={}
    
df_tweets['sentiment'] = df_tweets['text'].apply(lambda x: TextBlob(x).sentiment[0])
df_tweets['subjectivity'] = df_tweets['text'].apply(lambda x: TextBlob(x).sentiment[1])

In [ ]:
def subjectivity_filtering(df):
    print('filtering data using subjectivity detection')
    df=df[df['subjectivity']>0.4]
    return df

# Transform

In [ ]:
def transform(df):
    print('transforming data')
    
    # vectorize
    vectorizer = pickle.load(open("transformers/twitter/vectorizer.pickle", "rb"))
    feature_vector = vectorizer.transform(df['text'].astype(str))
    
    # defining variables
    X_test = feature_vector
    y_test = df['sentiment']
    
    # select features
    selector = pickle.load(open("transformers/twitter/selector.pickle", "rb"))
    X_test = selector.transform(X_test)
    
    return X_test, y_test

In [ ]:
def bert_predictions(df):
    # instantiate model
    tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
    model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
    
    # convert bert predictions between -1 and 1 as required for this comparative analysis
    bert_predictions_converter = {
        0:-1,
        1:-0.5,
        2:0,
        3:0.5,
        4:1
    }
    df['bert'] = df['text'].apply(lambda x:bert_predictions_converter[int(torch.argmax((model(tokenizer.encode(x, return_tensors='pt'))).logits))])

# Main

In [ ]:
df = df_tweets.copy()
print(len(df))

df = subjectivity_filtering(df)
print(len(df))

# function to generate bert sentiment predictions
bert_predictions(df)
    
# clean data
df = cleaning_data(df)
    
# transform into feature vector with selected features
X_test, y_test = transform(df)
    
# Evaluation
models = {}
# models['LR'] = joblib.load('models/final models/binary class/LR.pk1')
# models['MNB'] = joblib.load('models/final models/binary class/MNB.pk1')
# models['SVM'] = joblib.load('models/final models/binary class/SVM.pk1')
models['LR'] = joblib.load('models/twitter/LR_new.pk1')
models['MNB'] = joblib.load('models/twitter/MNB_new.pk1')
models['SVM'] = joblib.load('models/twitter/SVM_new.pk1')
models['MLP'] = joblib.load('models/twitter/MLP.pk1')
for model in models.keys():
    predictions = models[model].predict(X_test)
    df['predictions_{}'.format(model)] = predictions.tolist()

In [ ]:
df_final = df[['Timestamp','sentiment','bert','predictions_LR','predictions_MNB','predictions_SVM','predictions_MLP']]

In [ ]:
df_final = df_final.sort_values(by='Timestamp',ascending=True)

In [ ]:
polarity = {
    'positive':1,
    'negative':-1
}
rollingMean = 50
rollingMeanTB = 50
minPeriods = 10
df_final['predictions1_LR'] = df_final['predictions_LR'].apply(lambda x:polarity[x])
df_final['predictions1_MNB'] = df_final['predictions_MNB'].apply(lambda x:polarity[x])
df_final['predictions1_SVM'] = df_final['predictions_SVM'].apply(lambda x:polarity[x])
df_final['predictions1_MLP'] = df_final['predictions_MLP'].apply(lambda x:polarity[x])
df_final['rolling_LR'] = df_final.predictions1_LR.rolling(rollingMean,min_periods=minPeriods).mean()
df_final['rolling_MNB'] = df_final.predictions1_MNB.rolling(rollingMean,min_periods=minPeriods).mean()
df_final['rolling_SVM'] = df_final.predictions1_SVM.rolling(rollingMean,min_periods=minPeriods).mean()
df_final['rolling_MLP'] = df_final.predictions1_MLP.rolling(rollingMean,min_periods=minPeriods).mean()
df_final['rolling_TextBlob'] = df_final.sentiment.rolling(rollingMeanTB,min_periods=minPeriods).mean()
df_final['rolling_Bert'] = df_final.bert.rolling(rollingMean,min_periods=minPeriods).mean()

In [ ]:
# make 'Timestamp' the index
df_final.set_index('Timestamp',inplace=True)

In [ ]:
# delete duplicate rows where timestamp is same
df_final = df_final.loc[~df_final.index.duplicated(), :]

In [ ]:
# rolling sentiment plot
plt.figure(figsize=(14,10))
plt.axvspan(date2num(datetime(2019,12,12)), date2num(datetime(2019,12,12)), 
           label="Election Day",color="red", alpha=0.8)
plt.legend()
sns.lineplot(x='Timestamp',y='rolling_MLP',data=df_final,color='blue',label="MLP")
sns.lineplot(x='Timestamp',y='rolling_TextBlob',data=df_final,color='black',label="TextBlob")
sns.lineplot(x='Timestamp',y='rolling_Bert',data=df_final,color='brown',label="Bert")
plt.xlabel("Date",fontsize = 15)
plt.ylabel("Mean Sentiment",fontsize = 15)
plt.title('Jeremy Corbyn Twitter Account',fontsize = 20)
plt.gcf().autofmt_xdate()